<a href="https://colab.research.google.com/github/AnshMittal1811/BERT/blob/master/Keyphrase_Extraction_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Internship Assignment

In [1]:
!ls

sample_data


# Task 2
Using the model trained and built using Pranav's code, please extract key phrases the text file.

### Trying SciBERT according to the Instructions in BERT-keyphrase-extraction Repository

In [2]:
!git clone https://github.com/pranav-ust/BERT-keyphrase-extraction

Cloning into 'BERT-keyphrase-extraction'...
remote: Enumerating objects: 77, done.
remote: Total 77 (delta 0), reused 0 (delta 0), pack-reused 77
Unpacking objects: 100% (77/77), done.


In [3]:
!pip install pytorch-pretrained-BERT

     |████████████████████████████████| 133kB 4.6MB/s 


In [4]:
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar

--2020-06-06 21:20:23--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.184.48
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.184.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 410593280 (392M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 391.57M  48.0MB/s    in 8.0s    

2020-06-06 21:20:31 (48.8 MB/s) - ‘scibert_scivocab_uncased.tar’ saved [410593280/410593280]



In [5]:
!ls

BERT-keyphrase-extraction  sample_data	scibert_scivocab_uncased.tar


In [0]:
!mkdir BERT-keyphrase-extraction/model

In [7]:
!tar -C BERT-keyphrase-extraction/model -xvf scibert_scivocab_uncased.tar

scibert_scivocab_uncased/
scibert_scivocab_uncased/weights.tar.gz
scibert_scivocab_uncased/vocab.txt


In [0]:
import os
os.chdir('BERT-keyphrase-extraction/model/scibert_scivocab_uncased')

In [9]:
!ls

vocab.txt  weights.tar.gz


In [10]:
!tar -C ./ -xvf weights.tar.gz

bert_config.json
pytorch_model.bin


In [11]:
!ls

bert_config.json  pytorch_model.bin  vocab.txt	weights.tar.gz


In [0]:
!mv pytorch_model.bin ../
!mv vocab.txt ../
!mv bert_config.json ../

In [0]:
os.chdir('../../')

In [14]:
!ls

data		evaluate.py  metrics.py  README.md  utils.py
data_loader.py	experiments  model	 train.py


In [0]:
!rm -r model/scibert_scivocab_uncased

Here, we change the `params.json` file in `experiments/base_model` to increase number of epochs to 50 along with `min_epoch_num : 25` and `patience_num : 10`. 

In [18]:
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
loading archive file model/
Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

Weights of BertForTokenClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
Weights from pretrained model not used in BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
Starting training for 50 epoch(s)
Epoch 1/50
/usr/loca

In [19]:
!python evaluate.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model --restore_file best

Loading the dataset...
loading vocabulary file model/vocab.txt
- done.
Starting evaluation...
- Test metrics: loss: 00.63; f1: 46.45
             precision    recall  f1-score   support

          I      37.18     61.89     46.45       921

avg / total      37.18     61.89     46.45       921



Using trained and validated model to get keyphrases.

In [21]:
!git clone https://github.com/AnshMittal1811/BERT

Cloning into 'BERT'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [0]:
os.chdir('./BERT')

In [37]:
!ls

h1_7.txt  Keyphrase_Extraction_using_BERT.ipynb  README.md


In [0]:
!mv h1_7.txt ../BERT-keyphrase-extraction/

In [0]:
!rm -r BERT/

In [0]:
os.chdir('./BERT-keyphrase-extraction')

In [47]:
!ls

data		evaluate.py  h1_7.txt	 model	      README.md  utils.py
data_loader.py	experiments  metrics.py  __pycache__  train.py


In [68]:
!mkdir ./Keyphrase-Extract
!mv h1_7.txt ./Keyphrase-Extract
!ls

data		experiments	   model	train.py
data_loader.py	Keyphrase-Extract  __pycache__	utils.py
evaluate.py	metrics.py	   README.md


In [0]:
from pytorch_pretrained_bert import BertForTokenClassification, BertConfig, BertTokenizer
import random
import logging
import os
from metrics import f1_score
from metrics import classification_report
import numpy as np
import torch
from data_loader import DataLoader
import utils


### Prepare Dataset
Convert Semeval 2010 keyword recognition dataset to IO format dataset.

In [0]:
import os
from nltk.tokenize import sent_tokenize

We need to convert the file such that we can create a `tags.txt` file, similar to what has been created for `./data/task1/train`, `./data/task1/val`, `./data/task1/test`, and `./data/task1`. But, the facility for Semeval had been removed in AllenAI so we aren't able to calculate the IO format for the given text files without the archives.

In [76]:
# Load the parameters from json file
json_path = './experiments/base_model/params.json'
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = utils.Params(json_path)

# Use GPUs if available
params.device = torch.device('cuda' if torch.cuda.is_available() 
                                    else 'cpu')
params.n_gpu = torch.cuda.device_count()
params.multi_gpu = False

# Set the random seed for reproducible experiments
random.seed(23)
torch.manual_seed(23)
if params.n_gpu > 0:
    torch.cuda.manual_seed_all(23)  # set random seed for all GPUs
params.seed = 23

# Initialize the DataLoader (Problem) <----------- Problem over here (need IO formatted tags)
data_loader = DataLoader('./Keyphrase-Extract', './model', params, token_pad_idx=0)

# Load data 
test_data = data_loader.load_data('test')

# Specify the test set size
params.test_size = test_data['size']
params.eval_steps = params.test_size // params.batch_size
test_data_iterator = data_loader.data_iterator(test_data, shuffle=False)

config_path = './model/bert_config.json'
config = BertConfig.from_json_file(config_path)
model = BertForTokenClassification(config, num_labels=len(tag2idx))

model.to(params.device)

utils.load_checkpoint('experiments/base_model/best.pth.tar', model)


# Opening a file and reading a file
f = open("demofile.txt", "r")
input = []

for x in f:
  input.append(x)

# keyphrases extracted if the code works well
s = model(input)

FileNotFoundError: ignored

In [0]:
!rm -r model

In [78]:
!ls

data		evaluate.py  Keyphrase-Extract	__pycache__  train.py
data_loader.py	experiments  metrics.py		README.md    utils.py


In [0]:
os.chdir('./experiments/base_model')

In [0]:
!rm ./best.pth.tar
!rm ./evaluate.log
!rm ./last.pth.tar
!rm ./train.log

# Task 1 

Use the code in
https://github.com/pranav-ust/BERT-keyphrase-extraction to train it
with default parameters and evaluate the model for just Subtask 1 with
the following change: Instead of using SciBERT, please use BERT. As
you can see, Pranav has given evaluation results with SciBERT as F1
score: 0.6259; Precision: 0.5986; Recall: 0.6558; Support: 921. Please
give the results using BERT instead.

### Trying BERT according using BERT-keyphrase-extraction Repository

In [0]:
os.chdir('../../')
!mkdir ./model

In [82]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip

--2020-06-06 23:11:57--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip’

uncased_L-24_H-1024 100%[===================>]   1.16G   110MB/s    in 11s     

2020-06-06 23:12:08 (111 MB/s) - ‘uncased_L-24_H-1024_A-16.zip’ saved [1247797031/1247797031]



In [83]:
!unzip uncased_L-24_H-1024_A-16.zip -d ./model

Archive:  uncased_L-24_H-1024_A-16.zip
   creating: ./model/uncased_L-24_H-1024_A-16/
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: ./model/uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: ./model/uncased_L-24_H-1024_A-16/bert_config.json  


In [0]:
!rm -r uncased_L-24_H-1024_A-16.zip

In [0]:
os.chdir('./model/uncased_L-24_H-1024_A-16')

In [0]:
!mv bert_model.ckpt.meta ../
!mv bert_model.ckpt.data-00000-of-00001 ../
!mv bert_config.json ../
!mv vocab.txt ../
!mv bert_model.ckpt.index ../

In [0]:
!rm -r ../uncased_L-24_H-1024_A-16

In [0]:
os.chdir('/content/BERT-keyphrase-extraction/model')

In [89]:
!ls

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


#### Converting ckpt files to pytorch_model.bin

In [90]:
!pip install transformers

     |████████████████████████████████| 675kB 4.4MB/s 
     |████████████████████████████████| 890kB 11.7MB/s 
     |████████████████████████████████| 1.1MB 30.3MB/s 
     |████████████████████████████████| 3.8MB 40.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=83af3bc4c24f2e5ebd5d119c376f2f862ac78293396b68f4910302f9314e7b42
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [91]:
from transformers import BertConfig, BertForPreTraining, load_tf_weights_in_bert
import torch

PyTorch version 1.5.0+cu101 available.
TensorFlow version 2.2.0 available.


In [92]:
config = BertConfig.from_json_file('./bert_config.json')
print("Building PyTorch model from configuration: {}".format(str(config)))
model = BertForPreTraining(config)

# Load weights from tf checkpoint
load_tf_weights_in_bert(model, config, './bert_model.ckpt')

# Save pytorch-model
print("Save PyTorch model to {}".format('./pytorch_model.bin'))
torch.save(model.state_dict(), './pytorch_model.bin')

Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



Converting TensorFlow checkpoint from /content/BERT-keyphrase-extraction/model/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [1024]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 1024]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 1024]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_0/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_0/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_0/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_0/attention/self/key/kernel with shape [1024, 1024]
Loading TF wei

Save PyTorch model to ./pytorch_model.bin


In [0]:
!rm bert_model.ckpt.data-00000-of-00001
!rm bert_model.ckpt.index
!rm bert_model.ckpt.meta

In [0]:
os.chdir('../')

In [95]:
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
Traceback (most recent call last):
  File "train.py", line 168, in <module>
    train_data = data_loader.load_data('train')
  File "/content/BERT-keyphrase-extraction/data_loader.py", line 83, in load_data
    self.load_sentences_tags(sentences_file, tags_path, data)
  File "/content/BERT-keyphrase-extraction/data_loader.py", line 51, in load_sentences_tags
    sentences.append(self.tokenizer.convert_tokens_to_ids(tokens))
  File "/usr/local/lib/python3.6/dist-packages/pytorch_pretrained_bert/tokenization.py", line 121, in convert_tokens_to_ids
    ids.append(self.vocab[token])
KeyError: 'nonzero'


There is a difference in the vocabulary from the SciBERT models so, we use it instead for this dataset.

In [96]:
os.chdir('../')
!tar -C BERT-keyphrase-extraction/model -xvf scibert_scivocab_uncased.tar

scibert_scivocab_uncased/
scibert_scivocab_uncased/weights.tar.gz
scibert_scivocab_uncased/vocab.txt


In [0]:
os.chdir('BERT-keyphrase-extraction/model')
!rm vocab.txt
!mv ./scibert_scivocab_uncased/vocab.txt ./

In [0]:
!rm -r scibert_scivocab_uncased
os.chdir('../')

Now, we need to refer to use the recommended values of batch size of 4 and sequence length of 512, with 6 epochs, if GPU's VRAM is around 11 GB. (changed in experiments/base_model/params.json)
But, even after reducing that we still get CUDA out of Memory error so we use batch size of $2(changed)$ and sequence length of 512, with 50 epochs (`min_epoch_num:25` and `patience_num:10`). Learning rate = $3e^{-5}$(unchanged).

In [105]:
!python train.py --data_dir data/task1/ --bert_model_dir model/ --model_dir experiments/base_model

device: cuda, n_gpu: 1, 16-bits training: False
Loading the datasets...
loading vocabulary file model/vocab.txt
loading archive file model/
Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Weights of BertForTokenClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
Weights from pretrained model not used in BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
Starting training for

The above training and evaluations which is definitely flawed leads us to 22 interpretations which have been mentioned below.

* This flawed output ($f1 score = 0$) depicts that a `gridSearch` is needed for better Hyperparameter tuning and the bindings between SciBERT (produced by AllenAI) and BERT (google) need to be adjusted in order for the models to work on different codes. 
* Both models have different vocab list which leads to differences in both the implementations.
* Another solution is to use FastAI functions (such as `lr_find()` and `fit_one_cycle()`) to get better hyperparameters instead of gridSearchCV. This way is faster than generic GridSearchCV. These functions have been also used in computer vision task as can be seen here for [Wide ResNet22](https://github.com/AnshMittal1811/Pytorch/blob/master/Udemy%20-%20Pytorch%20zero%20to%20gans/L6%20ResNets%20and%20WRN22.ipynb).

-----